# Try different models-compare clustering results over eval. set
TODO: try different eps

In [1]:
# load eval.set (duplicates free)
import json
filename = 'pre-clustered_docs_harder.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)


## Title+abstract models
## TODO: lower case text

In [2]:
# train different models
from gensim.utils import simple_preprocess as sp
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]
# title + abstract models
train_corpus_ta = [ TaggedDocument(sp(doc['title'] + doc['abstract']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
epochs = 45
vec_size = 100
models = [
    # dm = 0, simple SG, simpler model, most of the time efficient and accurate
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 100'),
    Doc2Vec(dm=0, vector_size=500, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 500, lower eps'),
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800'),
    Doc2Vec(dm=0, vector_size=1000, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 1000'),
    # dm = 1, CBOW equivalent 
    Doc2Vec(dm=1, vector_size= 100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=100, alpha=0.05'),
    Doc2Vec(dm=1, vector_size= 150, min_count=2,epochs = epochs, 
            workers=multiprocessing.cpu_count(), alpha= 0.5, comment='dm=1, vec=150, alpha=0.5, hs=1'),
    Doc2Vec(dm=1, vector_size= 500, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=500, lower eps, min_pts=3'),
    Doc2Vec(dm=1, vector_size= 500, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=500, alpha=.05')
]
# also evaluate result on currently used model
loaded_model = Doc2Vec.load('/home/nick/anaconda3/bin/Tirocinio/doc2vec_tryouts/Models_Live_Test/d2v_abstract&title0.model')
print("Model loaded")

# build our vocabulary of words (all the unique words encountered inside our corpus)
for model in models:
    print(model)
    model.build_vocab(train_corpus_ta)
print("Vocabulary created!")

# train the models on the given data!
counter = 0
for model in models:
    print("Training %s" % model)
    %time model.train(train_corpus_ta, total_examples=len(train_corpus_ta), epochs=model.epochs)
    #model.save(MODEL_NAME+str(counter)+'.model')
    counter = counter + 1
#print("Models Saved")

Model loaded
Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4)
Doc2Vec("dm=0, vec = 500, lower eps",dbow,d500,n5,mc2,t4)
Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4)
Doc2Vec("dm=0, vec = 1000",dbow,d1000,n5,mc2,t4)
Doc2Vec("dm=1, vec=100, alpha=0.05",dm/m,d100,n5,w10,mc2,t4)
Doc2Vec("dm=1, vec=150, alpha=0.5, hs=1",dm/m,d150,n5,w5,mc2,s0.001,t4)
Doc2Vec("dm=1, vec=500, lower eps, min_pts=3",dm/m,d500,n5,w10,mc2,t4)
Doc2Vec("dm=1, vec=500, alpha=.05",dm/m,d500,n5,w10,mc2,t4)
Vocabulary created!
Training Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4)
CPU times: user 1.73 s, sys: 20 ms, total: 1.75 s
Wall time: 992 ms
Training Doc2Vec("dm=0, vec = 500, lower eps",dbow,d500,n5,mc2,t4)
CPU times: user 3.29 s, sys: 52 ms, total: 3.34 s
Wall time: 1.75 s
Training Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4)
CPU times: user 5.43 s, sys: 48 ms, total: 5.48 s
Wall time: 2.8 s
Training Doc2Vec("dm=0, vec = 1000",dbow,d1000,n5,mc2,t4)
CPU times: user 6.63 s, sys: 52 ms, total: 6.68 s
Wall time: 3

# Evaluate Clustering

In [15]:
import utils
import my_dbscan
import model_evaluation as me
import importlib
importlib.reload(me)

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples
for k, model in enumerate(models):
    # try different eps for some model
    if k == 1:
        eps = 0.11
        eps_increment = 0.13
        min_s = 2
    elif k==6:
        eps = 0.11
        eps_increment = 0.13
        min_s = 3
    else:
        eps = 0.11
        eps_increment = 0.13
        min_s = 2
    # get list of document vectors
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment,
                                               n_iterations = 3, verbose = False, min_samples = min_s)
    # get clusters as list of titles
    titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
    clusters_found.append(titles_clusters)
    #data = utils.plot_clusters(titles_clusters)
    
    # evaluate clustering
    expected_clusters = []
    for docs in cdocs:
        expected_clusters.append([doc['title'] for doc in docs])
    precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
    print('Precision score: %s, Recall score: %s'%(precision, recall))
    
    models_precision.append(precision)
    models_recall.append(recall)
    # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
    # that's way we have precision score
    print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
        len(urls_cluster_list), len(cdocs)))

## loaded model
inferred_vecs = [loaded_model.infer_vector(sp(doc['title']+doc['abstract'])) for doc in unwrapped_docs]
urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = inferred_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment, n_iterations = 3, verbose = False)
titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
clusters_found.append(titles_clusters)
# evaluate clustering
expected_clusters = []
for docs in cdocs:
    expected_clusters.append([doc['title'] for doc in docs])
precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
print('Precision score: %s, Recall score: %s'%(precision, recall))
models_precision.append(precision)
models_recall.append(recall)
print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
    len(urls_cluster_list), len(cdocs)))

Results occurences(correct guess, cluster index):  [(2, 13), (2, 1), (2, 2), (2, 9), (3, 10), (4, 5), (2, 13), (3, 6), (3, 11), (3, 11)]
Corrects/Length cluster they're in 2 5
Corrects/Length cluster they're in 2 2
Corrects/Length cluster they're in 2 2
Corrects/Length cluster they're in 2 2
Corrects/Length cluster they're in 3 3
Corrects/Length cluster they're in 4 4
Corrects/Length cluster they're in 2 5
Corrects/Length cluster they're in 3 3
Corrects/Length cluster they're in 3 6
Corrects/Length cluster they're in 3 6
Accuracy (Precision) over each cluster:  [40.0, 100.0, 100.0, 100.0, 100.0, 100.0, 40.0, 100.0, 50.0, 50.0]
Accuracy (Recall) over each cluster:  [50.0, 66.66666666666667, 66.66666666666667, 100.0, 30.0, 44.44444444444444, 66.66666666666667, 75.0, 30.0, 50.0]
Precision score: 78.0, Recall score: 57.94444444444446
#Number of clusters found: 15, against number of pre-computed clusters: 10#

Results occurences(correct guess, cluster index):  [(2, 11), (3, 1), (2, 2), (2, 

In [19]:
for cluster in clusters_found[3]:
    print(cluster)


['SpaceX: ecco come saranno le basi umane su Marte', 'Elon Musk contro la NASA per terraformare Marte']
['Decolla Facebook Dating, il Tinder di Menlo Park: primi test in Colombia - Repubblica.it', 'Facebook: Tinder nel mirino, novità di Instagram e Whatsapp - Corriere.it', 'Facebook Dating: ecco come funziona l’anti-Tinder di Zuckerberg - Corriere.it', 'Social e bambini: YouTube assume nuovi moderatori e Facebook lancia Messenger Kids - Corriere.it']
['Tesla sotto indagine per colpa dei tweet di Elon Musk: crollo in Borsa - Corriere.it', 'Elon Musk denuciato per truffa, Tesla crolla in borsa - Wired', 'Tesla, Elon Musk lascia la presidenza']
["L'equinozio d'autunno non è il 21 settembre: quest'anno arriva il 23 - Repubblica.it", "E' l'equinozio d'autunno - Spazio & Astronomia - ANSA.it"]
["iPhone Xs Max tira 3-4 volte più dell'Xs - Hi-tech - ANSA.it", 'Apple lancia iPhone Xs e la versione Max: sempre più grandi, gli smartphone sono la nuova Tv - Corriere.it']
['Fifa 19 contro Pes 2019:

# Visualize results graphically

In [17]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3', api_key='FdMB4O6qCfciGDOnLvdQ')
graph_name = 'model_eval_results'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since all the values are in between 0-1)
trace0 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_precision,
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x = model_descr + ['loaded model'],
    y = [s*100 for s in s_scores] + [0],
    name='Silhoutte score',
    marker=dict(
        color='rgb(204,50,100)',
    )
    
)

trace2 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_recall,
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    )
    
)
# precision, recall, silhoutte
data = [trace0, trace2,  trace1]
layout = go.Layout(
    title = 'Clustering Accuracy Results',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [16]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.27, eps_increment = 0.1, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.2057951
0.23791973
0.24932371
0.24040304
0.13758796
0.0041218703
0.12820731
0.228402


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.